<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Creditsafe - Get Company Credit Report

**Tags:** #creditsafe #api #enterprise #integrations #company #creditreport

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

**Description:** This notebook will demonstrate how to use the Creditsafe API to get a company credit report.

**References:**
- [Creditsafe API Documentation](https://www.creditsafe.com/gb/en/enterprise/integrations/api-documentation.html#tag/Companies/operation/companyCreditReport)

## Input

### Import libraries

In [1]:
import requests
import naas

### Setup Variables
- `username`: username used to connect to Creditsafe
- `password`: password used to connect to Creditsafe
- `env`: if 'sandbox' then we will change BASE URL
- `company_id`: company ID used in Creditsafe
- `siret`: SIRET to be set (if French company)

In [2]:
username = naas.secret.get("CREDITSAFE_USERNAME")
password = naas.secret.get("CREDITSAFE_PASSWORD")
env = "sandbox"
company_id = None
siret = "85272297400056"

## Model

### Create base url

In [3]:
if env == "sandbox":
    BASE_URL = f"https://connect.sandbox.creditsafe.com/v1"
else:
    BASE_URL = "https://connect.creditsafe.com/v1"    

### Get access token

In [17]:
def get_access_token(
    username,
    password
):
    # Init
    url = f"{BASE_URL}/authenticate"
    
    # Headers
    headers = {
        "Content type": "application/json"
    }
    
    # Payload
    data = {
        "username": username,
        "password": password,
    }
    
    # Request
    res = requests.post(url, headers=headers, data=data)
    res.raise_for_status
    if res.status_code == 200:
        return res.json().get("token")
    else:
        return None
    
access_token = get_access_token(username, password)
print(access_token)

### Get Company Credit Report

In [23]:
def get_company_report(
    access_token,
    company_id=None,
    siret=None
):
    if siret:
        company_id = f'FR-X-{siret}'
    # Init
    url = f"{BASE_URL}/companies/{company_id}?language=fr"
    
    # Headers
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    
    # Request
    res = requests.get(url, headers=headers)
    res.raise_for_status
    if res.status_code == 200:
        return res.json()
    else:
        return None
    
company_report = get_company_report(
    access_token,
    company_id=company_id,
    siret=siret
)

https://connect.sandbox.creditsafe.com/v1/companies/FR-X-85272297400056?language=fr
<Response [200]>


In [24]:
from pprint import pprint

report = company_report.get("report")
for r in report:
    print(r)

companyId
language
companySummary
companyIdentification
creditScore
contactInformation
shareCapitalStructure
directors
paymentData
negativeInformation
additionalInformation


In [20]:
pprint(report.get("companySummary"))

{'businessName': 'ACAI AND YOU',
 'companyNumber': 'FR34373758',
 'companyRegistrationNumber': '85272297400056',
 'companyStatus': {'description': 'Actif économiquement',
                   'isActive': True,
                   'providerStatus': 'Actif',
                   'status': 'Active'},
 'country': 'FR',
 'creditRating': {'commonDescription': 'Risque Faible',
                  'commonValue': 'B',
                  'creditLimit': {'currency': 'EUR', 'value': '20000'},
                  'pod': 0.00795,
                  'providerDescription': 'Risque faible',
                  'providerValue': {'maxValue': '100',
                                    'minValue': '1',
                                    'value': '51'}},
 'mainActivity': {'classification': 'NAF/APE',
                  'code': '5610C',
                  'description': 'Restauration de type rapide'}}


In [26]:
pprint(report.get("creditScore"))

{'currentCreditRating': {'commonDescription': 'Risque Faible',
                         'commonValue': 'B',
                         'creditLimit': {'currency': 'EUR', 'value': '20000'},
                         'pod': 0.00795,
                         'providerDescription': 'Risque faible',
                         'providerValue': {'maxValue': '100',
                                           'minValue': '1',
                                           'value': '51'}},
 'latestRatingChangeDate': '2023-02-15T00:00:00Z',
 'previousCreditRating': {'commonDescription': 'Risque Elevé',
                          'commonValue': 'D',
                          'creditLimit': {'currency': 'EUR', 'value': '0'},
                          'pod': 0.101,
                          'providerDescription': 'Risque très Elevé',
                          'providerValue': {'maxValue': '100',
                                            'minValue': '1',
                                            'value': '1

## Output

### Display result

In [ ]:
print(company_credit_report)